In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install --upgrade keras

In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tf_keras


In [ ]:
!pip show keras

Name: keras
Version: 3.3.3
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow


In [ ]:
import os
import librosa
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, MaxPool2D, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GroupKFold


# Function to extract spectrograms and resize them
def extract_and_resize_spectrogram(audio_file, target_shape=(128, 128)):
    y, sr = librosa.load(audio_file, sr=22050)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    resized_spectrogram = np.resize(spectrogram, target_shape)
    return resized_spectrogram

# Path to the GTZAN dataset folders (each class label has its own folder)
gtzan_path = '/content/drive/MyDrive/Datasets/Music_Genre_Classification/genres'

# Initialize arrays
data_array = []
label_array = []
group_array = []

# Iterate through each class label folder
class_folders = sorted(os.listdir(gtzan_path))
for class_label, class_folder in enumerate(class_folders):
    class_folder_path = os.path.join(gtzan_path, class_folder)
    audio_files = os.listdir(class_folder_path)
    for audio_file in audio_files:
        audio_file_path = os.path.join(class_folder_path, audio_file)
        spectrogram = extract_and_resize_spectrogram(audio_file_path)
        data_array.append(spectrogram)
        label_array.append(class_label)
        group_array.append(class_label)

# Convert arrays to NumPy arrays
data_array = np.array(data_array)
label_array = np.array(label_array)
group_array = np.array(group_array)

# Define the 1D CNN model
def cnnmodel_tuned(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling2D())
    model.add(Dense(10, activation='softmax'))  # Assuming 10 classes
    return model



# Initialize GroupKFold
gkf = GroupKFold(n_splits=3)


# Example usage of GroupKFold with CNN model training
accuracy = []
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features, train_labels = data_array[train_index], label_array[train_index]
    val_features, val_labels = data_array[val_index], label_array[val_index]

    # Reshape data for 1D CNN input
    train_features = np.expand_dims(train_features, axis=-1)
    val_features = np.expand_dims(val_features, axis=-1)

    # Create and compile the CNN model
    model = cnnmodel_tuned(input_shape=train_features.shape[1:])
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(train_features, train_labels, epochs=100, batch_size=32, validation_data=(val_features, val_labels))

    # Evaluate the model
    _, acc = model.evaluate(val_features, val_labels)
    accuracy.append(acc)



KeyboardInterrupt: 

In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import GroupKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, MaxPool2D, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D, Flatten
from tensorflow.keras.optimizers import Adam

# Function to extract spectrograms and resize them
def extract_and_resize_spectrogram(audio_file, target_shape=(128, 128)):
    y, sr = librosa.load(audio_file, sr=22050)
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    resized_spectrogram = np.resize(spectrogram, target_shape)
    return resized_spectrogram

# Path to the GTZAN dataset folders (each class label has its own folder)
gtzan_path = '/content/drive/MyDrive/Datasets/Music_Genre_Classification/genres'

# Initialize arrays
data_array = []
label_array = []
group_array = []

# Iterate through each class label folder
class_folders = sorted(os.listdir(gtzan_path))
for class_label, class_folder in enumerate(class_folders):
    class_folder_path = os.path.join(gtzan_path, class_folder)
    audio_files = os.listdir(class_folder_path)
    for audio_file in audio_files:
        audio_file_path = os.path.join(class_folder_path, audio_file)
        spectrogram = extract_and_resize_spectrogram(audio_file_path)
        data_array.append(spectrogram)
        label_array.append(class_label)
        group_array.append(class_label)

# Convert arrays to NumPy arrays
data_array = np.array(data_array)
label_array = np.array(label_array)
group_array = np.array(group_array)

# Define the CNN model
def cnnmodel_tuned(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling2D())
    model.add(Dense(10, activation='softmax'))  # Assuming 10 classes
    return model

# Initialize GroupKFold
gkf = GroupKFold(n_splits=3)

# Example usage of GroupKFold with CNN model training
accuracy = []
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features, train_labels = data_array[train_index], label_array[train_index]
    val_features, val_labels = data_array[val_index], label_array[val_index]

    # Reshape data for CNN input
    train_features = np.expand_dims(train_features, axis=-1)
    val_features = np.expand_dims(val_features, axis=-1)

    # Create and compile the CNN model
    model = cnnmodel_tuned(input_shape=train_features.shape[1:])
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(train_features, train_labels, epochs=100, batch_size=32, validation_data=(val_features, val_labels))

    # Evaluate the model
    _, acc = model.evaluate(val_features, val_labels)
    accuracy.append(acc)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.2676 - loss: 1.9871 - val_accuracy: 0.0000e+00 - val_loss: 6.0444
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.4163 - loss: 1.4435 - val_accuracy: 0.0000e+00 - val_loss: 7.3887
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.4812 - loss: 1.3526 - val_accuracy: 0.0000e+00 - val_loss: 7.0030
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.5131 - loss: 1.2930 - val_accuracy: 0.0000e+00 - val_loss: 6.7602
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5281 - loss: 1.2069 - val_accuracy: 0.0000e+00 - val_loss: 6.2975
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.5003 - loss: 1.2494 - val_accuracy: 0.0000e+00 - val_loss: 6.1972
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.5566 - loss: 1.1895 - val_accuracy: 0.0000e+00 - val_loss: 6.2248
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.6036 - loss: 1.123

In [ ]:
# Print average accuracy
print("Average accuracy:", accuracy)

Average accuracy: [0.0, 0.0, 0.0]


In [ ]:

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_10 (LeakyReLU)           │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 61, 61, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_4                  │ (None, 14, 14, 128)         │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 12, 12, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_5                  │ (None, 6, 6, 256)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 4,719,648 (18.00 MB)

 Trainable params: 1,573,194 (6.00 MB)

 Non-trainable params: 64 (256.00 B)

 Optimizer params: 3,146,390 (12.00 MB)

In [ ]:
pip install --upgrade scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4


In [ ]:

import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score


# Function to extract time-domain waveforms from audio files with fixed length
def extract_waveform(audio_file, label_index, duration=30, target_length=44100):
    y, sr = librosa.load(audio_file, duration=duration, sr=None)  # Load audio file (full duration, original sampling rate)
    # Trim or pad the waveform to the target length
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)), mode='constant')
    elif len(y) > target_length:
        y = y[:target_length]
    return y, label_index

# Define paths to GTZAN dataset and class labels
gtzan_path = '/content/drive/MyDrive/Datasets/Music_Genre_Classification/genres'
class_labels = os.listdir(gtzan_path)

# Initialize lists to store waveforms and labels
X = []
y = []

# Iterate through each class label folder
for label_index, label in enumerate(class_labels):  # Use enumerate to get the index of each label
    class_folder = os.path.join(gtzan_path, label)
    for audio_file in os.listdir(class_folder):
        audio_file_path = os.path.join(class_folder, audio_file)
        waveform, label_index = extract_waveform(audio_file_path, label_index)
        X.append(waveform)
        y.append(label_index)  # Append the label index directly

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for Conv1D layer
input_shape = (44100, 1)  # Shape is (timesteps, input_dim)

# Build the 1D CNN model
def cnnmodel_tuned(input_shape):
    model = Sequential()
    # Convolutional layers
    model.add(Conv1D(filters=32, kernel_size=3, strides=1, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))

    model.add(Conv1D(filters=64, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(filters=128, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(filters=256, kernel_size=3, strides=1))
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))

    # Global pooling and dense layers
    model.add(GlobalAveragePooling1D())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # Assuming 10 classes

    return model


# Define the model
model = cnnmodel_tuned(input_shape)
# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Predict on test data
y_pred = model.predict_classes(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy (from predictions): {test_accuracy:.4f}")


Epoch 1/50
25/25 [==============================] - 219s 9s/step - loss: 2.3080 - accuracy: 0.0776 - val_loss: 2.3011 - val_accuracy: 0.1450
Epoch 2/50
25/25 [==============================] - 199s 8s/step - loss: 2.2719 - accuracy: 0.1039 - val_loss: 2.3010 - val_accuracy: 0.1300
Epoch 3/50
25/25 [==============================] - 217s 9s/step - loss: 2.2612 - accuracy: 0.1139 - val_loss: 2.3016 - val_accuracy: 0.1050
Epoch 4/50
25/25 [==============================] - 202s 8s/step - loss: 2.2510 - accuracy: 0.1364 - val_loss: 2.3021 - val_accuracy: 0.0850
Epoch 5/50
25/25 [==============================] - 219s 9s/step - loss: 2.2360 - accuracy: 0.1652 - val_loss: 2.3045 - val_accuracy: 0.0800
Epoch 6/50
25/25 [==============================] - 214s 9s/step - loss: 2.2109 - accuracy: 0.1852 - val_loss: 2.3058 - val_accuracy: 0.0750
Epoch 7/50
25/25 [==============================] - 208s 8s/step - loss: 2.1692 - accuracy: 0.2403 - val_loss: 2.3114 - val_accuracy: 0.0750
Epoch 8/50
25

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Function to extract time-domain waveforms from audio files with fixed length
def extract_waveform(audio_file, label_index, duration=30, target_length=44100):
    y, sr = librosa.load(audio_file, duration=duration, sr=None)  # Load audio file (full duration, original sampling rate)
    # Trim or pad the waveform to the target length
    if len(y) < target_length:
        y = np.pad(y, (0, target_length - len(y)), mode='constant')
    elif len(y) > target_length:
        y = y[:target_length]
    return y, label_index

# Define paths to GTZAN dataset and class labels
gtzan_path = '/content/drive/MyDrive/Datasets/Music_Genre_Classification/genres'
class_labels = os.listdir(gtzan_path)

# Initialize lists to store waveforms and labels
X = []
y = []

# Iterate through each class label folder
for label_index, label in enumerate(class_labels):  # Use enumerate to get the index of each label
    class_folder = os.path.join(gtzan_path, label)
    for audio_file in os.listdir(class_folder):
        audio_file_path = os.path.join(class_folder, audio_file)
        waveform, label_index = extract_waveform(audio_file_path, label_index)
        X.append(waveform)
        y.append(label_index)  # Append the label index directly

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for Conv2D layer
input_shape = (44100, 1, 1)  # Shape is (timesteps, input_dim, channels)

# Expand dimensions to match the input shape
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Build the 2D CNN model
def cnnmodel_tuned(input_shape):
    model = Sequential()
    # Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1)))
    model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Flatten the output before dense layers
    model.add(Flatten())

    # Dense layers
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))  # Assuming 10 classes

    return model

# Define the model
model = cnnmodel_tuned(input_shape)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Predict on test data
y_pred = model.predict_classes(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy (from predictions): {test_accuracy:.4f}")
